# 문서 검색에서의 설명 가능한 검색 (Explainable Retrieval)

## 개요

이 코드는 설명 가능한 검색 시스템(Explainable Retriever)을 구현하여, 쿼리에 기반해 관련 문서를 검색할 뿐만 아니라 **각 검색 결과에 대한 설명**도 제공합니다. 벡터 기반 유사성 검색과 자연어 설명을 결합하여 검색 과정의 투명성과 해석 가능성을 높이는 것이 목표입니다.

## 동기

기존 문서 검색 시스템은 종종 결과를 제공하지만, **선택 이유에 대한 설명이 부족한 블랙 박스**로 작동하는 경우가 많습니다. 이로 인해 결과를 이해하는 것이 중요한 시나리오에서는 문제가 될 수 있습니다. 설명 가능한 검색 시스템은 **각 검색 결과가 왜 관련성이 있는지를 사용자에게 설명**하여 투명성과 이해도를 높입니다.

## 주요 구성 요소

1. 입력 텍스트로부터 벡터 스토어 생성
2. 효율적인 유사성 검색을 위한 FAISS 기반의 기본 검색기
3. 설명을 생성하는 언어 모델(LLM)
4. 검색 및 설명 생성을 결합한 `ExplainableRetriever` 클래스

## 방법 설명

### 문서 전처리 및 벡터 스토어 생성

1. 입력 텍스트를 OpenAI 임베딩 모델을 사용해 임베딩으로 변환합니다.
2. FAISS 벡터 스토어를 생성하여 유사성 검색의 효율성을 높입니다.

### 검색기 설정

1. 벡터 스토어를 기반으로 기본 검색기를 생성하며, 상위 5개의 유사한 문서를 반환하도록 구성합니다.

### 설명 생성

1. GPT-4 모델과 같은 LLM을 사용하여 검색 결과에 대한 설명을 생성합니다.
2. 검색 결과가 쿼리와 관련이 있는 이유를 설명하도록 LLM에 지시하는 맞춤 프롬프트 템플릿을 정의합니다.

### ExplainableRetriever 클래스"

1. 기본 검색기와 설명 생성을 결합한 단일 인터페이스를 제공합니다.
2. `retrieve_and_explain` 메서드:
   - 기본 검색기를 통해 관련 문서를 검색합니다.
   - 각 검색된 문서에 대해 쿼리와의 관련성을 설명합니다.
   - 문서 내용과 설명이 담긴 사전 리스트를 반환합니다.

## 이 접근 방식의 이점

1. **투명성**: 사용자는 특정 문서가 검색된 이유를 이해할 수 있습니다.
2. **신뢰도 향상**: 설명을 통해 시스템 결과에 대한 사용자 신뢰도가 높아집니다.
3. **학습 효과**: 사용자에게 쿼리와 문서 간의 관계에 대한 인사이트를 제공합니다.
4. **디버깅 용이**: 검색 과정에서 발생하는 문제를 식별하고 수정하기 쉬워집니다.
5. **맞춤화 가능**: 설명 프롬프트를 사용 사례나 분야에 맞게 조정할 수 있습니다.

## 결론

설명 가능한 검색 시스템은 **더 해석 가능하고 신뢰할 수 있는 정보 검색 시스템**을 만드는 중요한 단계입니다. 검색된 문서와 함께 자연어 설명을 제공함으로써 강력한 벡터 기반 검색 기술과 인간의 이해 사이의 간극을 좁힙니다. 법률 연구, 의료 정보 시스템, 교육 도구와 같이 **검색의 이유가 중요한 분야**에서 활용될 잠재력을 가지고 있습니다.


 ### Import libraries

In [1]:
import os
import sys
from dotenv import load_dotenv


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_6911/2996950230.py:7: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import *
/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.4.5, however version 1.4.7 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


### Define the explainable retriever class 

In [2]:
class ExplainableRetriever:
    def __init__(self, texts):
        self.embeddings = OpenAIEmbeddings()

        self.vectorstore = FAISS.from_texts(texts, self.embeddings)
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)

        
        # top5개 관련 문서 추출하는 리트리버
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 5})
        
        # 쿼리와 반환된 문서들의 관계를 분석한다.
        # 왜 해당 반환된 문서가 커리와 연관있는지 , 얼마나 쿼리를 답변하는데 도움을 주는지 설명하는 프롬프트 
        explain_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template="""
            Analyze the relationship between the following query and the retrieved context.
            Explain why this context is relevant to the query and how it might help answer the query.
            
            Query: {query}
            
            Context: {context}
            
            Explanation:
            """
        )
        self.explain_chain = explain_prompt | self.llm

    def retrieve_and_explain(self, query):
        # 쿼리와 관련된 문서 5개 추출 
        docs = self.retriever.get_relevant_documents(query)
        
        explained_results = []
        
        # 쿼리와 문맥이 들어가고 설명을 함 
        for doc in docs:
            # Generate explanation
            input_data = {"query": query, "context": doc.page_content}
            explanation = self.explain_chain.invoke(input_data).content
            # 리스트에 반환된 문서와 설명을 담아서 내보냄 
            explained_results.append({
                "content": doc.page_content,
                "explanation": explanation
            })
        
        return explained_results



### Create a mock example and explainable retriever instance

In [3]:

# Usage
texts = [
    "The sky is blue because of the way sunlight interacts with the atmosphere.",
    "Photosynthesis is the process by which plants use sunlight to produce energy.",
    "Global warming is caused by the increase of greenhouse gases in Earth's atmosphere."
]

explainable_retriever = ExplainableRetriever(texts)


### Show the results

In [5]:
query = "Why is the sky blue?"
# 쿼리에 대한 5가지 문서에 대해 왜 추출했는지 설명하는 결과
results = explainable_retriever.retrieve_and_explain(query)

for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"Content: {result['content']}")
    print(f"Explanation: {result['explanation']}")
    print()

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_6911/1069776825.py:30: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = self.retriever.get_relevant_documents(query)


Result 1:
Content: The sky is blue because of the way sunlight interacts with the atmosphere.
Explanation: The context provided directly addresses the query by explaining the reason behind the blue color of the sky. It highlights the interaction between sunlight and the atmosphere, which is a fundamental aspect of why we perceive the sky as blue.

This context is relevant to the query for several reasons:

1. **Direct Answer**: The context provides a straightforward answer to the question, indicating that the phenomenon is due to the interaction of sunlight with atmospheric elements.

2. **Scientific Basis**: The mention of sunlight and the atmosphere suggests a scientific explanation, which is likely what the person asking the question is seeking. It implies that the answer is grounded in physics and atmospheric science.

3. **Foundation for Further Exploration**: While the context is concise, it opens the door for further exploration into related topics, such as Rayleigh scattering, 

In [12]:
# 왜 이렇게 retrive했는지에 대한 설명 
results[1]['explanation'].split('\n\n')

["The context provided, which discusses global warming and greenhouse gases, is not directly relevant to the query about why the sky is blue. The query seeks an explanation of the phenomenon of the blue sky, which is primarily due to Rayleigh scattering of sunlight by the Earth's atmosphere. ",
 'To answer the query effectively, relevant context would include information about the scattering of light, the composition of the atmosphere, and how different wavelengths of light are affected by this scattering. ',
 'While the context about global warming touches on atmospheric conditions, it does not address the specific scientific principles that explain the color of the sky. Therefore, it does not help answer the query about why the sky appears blue. ',
 'In summary, the context is not relevant to the query, as it does not provide the necessary information to explain the phenomenon in question.']

## 내 생각 
- 반환된 문서에 대해 왜 반환되었는지 설명을 하도록 llm에게 역할을 부여함 
- 해당 설명을 통해 해석가능하고 신뢰할 수 있는 검색 시스템을 만든는 것
- 이해를 시켜주는 것 -> 법률, 의료와 같이 검색의 이유가 중요할 때 활용됨 